# notepad

## Imports

The data will be processed using 

In [1]:
import os       # using operating system dependent functionality (folders)
import glob
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
#import SimpleITK as sitk
import skimage
import hashlib
import sys
sys.path.insert(0, '../') # path to functions
from brainspin import file_handler as fh # 
from brainspin import mold #
from brainspin import carve
from brainspin.file_handler import Config

## dicom stripping

In [7]:
from brainspin.carve import PydicomDicomReader

In [9]:
dicomfile_directory1 = "../tests/dicom_example_folder"
source_column = 'file'
reader = PydicomDicomReader(
        exclude_fields=('PatientName',),
    )
source = carve.DirectorySource(dicomfile_directory1, source_column)
df = reader.read(source)
df

,file,PhotometricInterpretation,SeriesDescription,SpecificCharacterSet,StudyID,PatientID,ActualFrameDuration,ReferringPhysicianName,SeriesTime,SeriesNumber,...,ImagesInAcquisition,PatientAge,AcquisitionDate,BitsStored,CorrectedImage,HighBit,StudyTime,InstanceNumber,NumberOfDetectors,DetectorVector
0,IM-0001-0002.dcme3cbbe6b-2d21-4798-a27a-10226c...,MONOCHROME2,Thyroid Tc,ISO_IR 100,1,SIEMENS,132121,(),153319.140000,1,...,3,051Y,20010131,16,UNIF,15,114808.390000,2,1,1
1,IM-0001-0003.dcma8cf5740-b518-4e91-be5e-3b6cdb...,MONOCHROME2,Thyroid Tc,ISO_IR 100,1,SIEMENS,131348,(),153319.140000,1,...,3,051Y,20010131,16,UNIF,15,114808.390000,3,1,1
2,IM-0001-0004.dcm0a117e34-be84-4fa8-993d-249927...,MONOCHROME2,Post syringe,ISO_IR 100,1,SIEMENS,60003,(),152842.140000,1,...,1,051Y,20010131,16,UNIF,15,114808.390000,1,1,1


In [14]:
output_folder = 'test'
try: 
    os.mkdir(output_folder)
except FileExistsError:
    pass 

jpg = PydicomDicomReader.rip_out_jpgs(dicomfile_directory1,source,output_folder)

..\brainspin\carve.py:230: UserWarning: test\IM-0001-0002.dcme3cbbe6b-2d21-4798-a27a-10226c1ccc3e.jpg is a low contrast image
  io.imsave(
Lossy conversion from uint16 to uint8. Losing 8 bits of resolution. Convert image to uint8 prior to saving to suppress this warning.
..\brainspin\carve.py:230: UserWarning: test\IM-0001-0003.dcma8cf5740-b518-4e91-be5e-3b6cdbd43872.jpg is a low contrast image
  io.imsave(
Lossy conversion from uint16 to uint8. Losing 8 bits of resolution. Convert image to uint8 prior to saving to suppress this warning.
..\brainspin\carve.py:230: UserWarning: test\IM-0001-0004.dcm0a117e34-be84-4fa8-993d-249927b41af9.jpg is a low contrast image
  io.imsave(
Lossy conversion from uint16 to uint8. Losing 8 bits of resolution. Convert image to uint8 prior to saving to suppress this warning.


In [ ]:

def hash_folder(origin_folder1, file_extension, made, force=False):
    """Hashing function to be used by command line.

    :param origin_folder1: The string of the folder with files to hash
    :type origin_folder1: str
    :param file_extension: File extension
    :type file_extension: str
    :param made: file directory where csv with hashes will be put
    :type made: str
    """
    filepath = os.path.join(made, 'hash_output.csv')
    df = hash_rash(origin_folder1, file_extension)
    if not force:
        if os.path.isfile(filepath):
            return
    try:
        os.makedirs(os.path.dirname(filepath))
    except FileExistsError:
        pass

    df.to_csv(filepath)

# def save_preprocessed(array, out_fname, force):
#     """
#     This function is written to be called by the cli module.
#     It stores arrays in a directory.
#     """
#     if not force:
#         if os.path.isfile(out_fname):
#             return
#     try:
#         os.makedirs(os.path.dirname(out_fname))
    # except FileExistsError:
    #     pass
    # np.save(out_fname, array, allow_pickle=False)


def hash_rash(origin_folder1, file_extension):
    """Hashing function to check files are not corrupted or to assure
    files are changed.

    :param origin_folder1: The string of the folder with files to hash
    :type origin_folder1: str
    :param file_extension: File extension
    :type file_extension: str

    :returns: Dataframe with hashes for what is in folder
    :rtype: ~pandas.DataFrame
    """
    hash_list = []
    file_names = []
    files = '**/*.' + file_extension
    
    non_suspects1 = glob.glob(
        os.path.join(origin_folder1, files),
        recursive=True,
    )
    # print(non_suspects1)
    BUF_SIZE = 65536
    for file in non_suspects1:
        sha256 = hashlib.sha256()
        with open(file, 'rb') as f:
            while True:
                data = f.read(BUF_SIZE)
                if not data:
                    break
                sha256.update(data)
        result = sha256.hexdigest()
        hash_list.append(result)
        file_names.append(file)
        #print(file_names)
    df = pd.DataFrame(hash_list, file_names)
    df.columns = ["hash"]
    df = df.reset_index()
    df = df.rename(columns={'index': 'file_name'})
    #df.to_csv('out.csv')
    print(df)

    return df


In [ ]:
def return_2DF():

    date = pd.date_range('today', periods=20)
    DF1 = pd.DataFrame(np.random.rand(20, 2), index=date)

    #DF2 = pd.DataFrame(np.random.rand(20, 4), index=date, columns='A B C D'.split())

    return DF1

In [ ]:
DF1 = return_2DF()
DF1

In [ ]:
fh.hash_folder('../not_pushed', 'gz', 'hope')

In [ ]:
print(output)

In [ ]:
pd.read_csv('out.csv')

In [ ]:
file_directory = '../not_pushed'
file_directory_list = glob.glob(
    os.path.join(file_directory, '**/*.tsv'),
    recursive=True,
    )
for file in file_directory_list:
            print(file)

In [ ]:
# non_suspects1 = glob.glob(os.path.join('../not_pushed','**/*.tsv' ))
# print(non_suspects1)
for file in files:
        if file.endswith('.tsv'):
            print(file)

## Load files

Use the config pathways for the different datasets

In [ ]:
config = Config()
root_mri_directory = config.get_directory('root_mri_directory')

In [ ]:
tsv_pattern = os.path.join(root_mri_directory, '**/*.tsv')
tsv_files = glob.glob(tsv_pattern, recursive=True)


In [ ]:
tsv_files

In [ ]:
dataframe_example = pd.read_csv(tsv_files[0], sep='\t')

In [ ]:
dataframe_example

In [ ]:
mri_pattern = os.path.join(root_mri_directory, '**/*.gz')
gz_files = glob.glob(mri_pattern, recursive=True)

In [ ]:
gz_files

In [ ]:
# here we can use something not in the base environment just to check that these files exist correctly



# A path to an mrid brain .nii image:
t1_fn = gz_files[0]

# Read the .nii image containing the volume with SimpleITK:
sitk_t1 = sitk.ReadImage(t1_fn)

# and access the numpy array:
t1 = sitk.GetArrayFromImage(sitk_t1)

# now display it

In [ ]:
t1.shape

In [ ]:
# import plotly
# import plotly.express as px


# fig = px.imshow(
#     t1,
#     facet_col=1,
#     animation_frame=0,
#     binary_string=True,
#     binary_format="jpg",
# )
# fig.layout.annotations[0]["text"] = "Something0"
# fig.layout.annotations[1]["text"] = "Something2"
# plotly.io.show(fig)

In [ ]:
sitk.Show(sitk_t1, debugOn=True)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
data = t1
z, x, y = data.nonzero()
ax.scatter(x, y, z, c=z, alpha=1)
plt.show()